# VAE

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import pandas as pd
import utils
from utils import PROJECT_ROOT, DATA_PATH
import yuval_module.paper_source as PaperSource
%load_ext autoreload
%autoreload 2

In [2]:
#sttings
batch_size= 8
epochs= 2
cuda= torch.cuda.is_available()
seed= 42
log_interval= 10
num_workers= 2

In [3]:
#check for cuda
device = torch.device('cuda' if cuda else 'cpu')
device

device(type='cpu')

In [4]:
torch.manual_seed(seed)


In [5]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

In [6]:
from torch.utils.data import Dataset, DataLoader

In [7]:
import py_4.get_mesh_vec as get_mesh_vec

In [8]:
mesh_embed=get_mesh_vec.MeshEmbeddings()

In [9]:
FILE = "enriched_labeled_dataset_large.json" 
if os.path.exists(PROJECT_ROOT + DATA_PATH + FILE):
    print("READING FROM LOCAL")
    if FILE.split(".")[1] == "json":
        df = pd.read_json(PROJECT_ROOT + DATA_PATH + FILE)
    else:
        df = pd.read_csv(PROJECT_ROOT + DATA_PATH + FILE)
    #ps = PaperSource()
else:
    print("PULLING FROM S3")
    ps = sim_matrix_3.load_dataset("enriched_labeled_dataset")
    df = ps.get_dataset()

df.drop(columns=["last_author_country"],inplace=True)
df.rename(columns={'ORG_STATE':'last_author_country'},inplace=True)

print("FILE PULLED")

READING FROM LOCAL
FILE PULLED


In [11]:
mesh_embed.get_feat_mesh(df[:20].mesh.to_list()).shape

MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male


(20, 64)

In [12]:
selection=['8352708','6913641']
PI_lst=df.PI_IDS[df.PI_IDS.isin(selection)].to_list()

In [13]:
class ToyDS(Dataset):
    def __init__(self,df):
        super().__init__()
        self.df = mesh_embed.get_feat_mesh(df[:1000].mesh.to_list())
        #self.df = mesh_embed.get_feat_mesh(df[df.PI_IDS.isin(selection)].mesh.to_list()) 

        
 
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        features= self.df[idx]
        return features, features

In [14]:
train_set = ToyDS(df)
train_loader=DataLoader(dataset= train_set, batch_size = batch_size, shuffle = True, num_workers = num_workers)

MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAM

In [15]:
class VAE(nn.Module):
    def __init__(self, input_dim = 64):
        super(VAE, self).__init__()
        self.input_dim = input_dim
        self.fc1 = nn.Linear(self.input_dim, 32)
        self.fc21 = nn.Linear(32, 8)
        self.fc22 = nn.Linear(32, 8)
        self.fc3 = nn.Linear(8, 32)
        self.fc4 = nn.Linear(32, self.input_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, self.input_dim))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar, z

In [16]:
def loss_function(recon_x, x, mu, logvar):
    #BCE = F.binary_cross_entropy(recon_x, x.view(-1, 64), reduction='sum')

    MSE = F.mse_loss(recon_x, x.view(-1, 64))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return MSE + KLD

In [18]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [19]:
from torchsummary import summary
summary(model, (20,64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 32]           2,080
            Linear-2                    [-1, 8]             264
            Linear-3                    [-1, 8]             264
            Linear-4                   [-1, 32]             288
            Linear-5                   [-1, 64]           2,112
Total params: 5,008
Trainable params: 5,008
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.02
Estimated Total Size (MB): 0.03
----------------------------------------------------------------


In [20]:
import numpy as np

In [21]:
def train_epoch(tr_loader, model, criterion, optimizer, num_epochs):
    model.train()
    train_log = []
    
    for epoch in range(num_epochs):
        bottle_neck=[]
        print("Started training epoch no. {}",format(epoch+1))
        train_loss= 0
        for batch_idx, (data, _) in enumerate(train_loader):
            data = data.to(device, dtype=torch.float32)

            optimizer.zero_grad()
            recon_batch, mu, logvar, z= model(data)
            bottle_neck.extend(z.detach().numpy())
            loss = criterion(recon_batch, data, mu, logvar)
            loss.backward()
            train_loss += loss.item()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader),
                    loss.item() / len(data)))

        print('====> Epoch: {} Average loss: {:.4f}'.format(
            epoch, train_loss / len(train_loader.dataset)))
        train_log.append({
            'epoch': epoch,
            'train_loss': train_loss / len(train_loader)})
    return train_log, bottle_neck

In [97]:
# def test(epoch):
#     model.eval()
#     test_loss = 0
#     with torch.no_grad():
#         for i, (data, _) in enumerate(test_loader):
#             data = data.to(device)
#             recon_batch, mu, logvar = model(data)
#             test_loss += loss_function(recon_batch, data, mu, logvar).item()
#             if i == 0:
#                 n = min(data.size(0), 8)
#                 comparison = torch.cat([data[:n],
#                                       recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
#                 save_image(comparison.cpu(),
#                          'results/reconstruction_' + str(epoch) + '.png', nrow=n)

#     test_loss /= len(test_loader.dataset)
#     print('====> Test set loss: {:.4f}'.format(test_loss))

In [22]:
train_log, bottle_neck= train_epoch(train_loader,model,loss_function,optimizer,num_epochs= 3)

Started training epoch no. {} 1
Train Epoch: 0 [0/1000 (0%)]	Loss: 0.120979
Train Epoch: 0 [80/1000 (8%)]	Loss: 0.080456
Train Epoch: 0 [160/1000 (16%)]	Loss: 0.058940
Train Epoch: 0 [240/1000 (24%)]	Loss: 0.054971
Train Epoch: 0 [320/1000 (32%)]	Loss: 0.048100
Train Epoch: 0 [400/1000 (40%)]	Loss: 0.045853
Train Epoch: 0 [480/1000 (48%)]	Loss: 0.037555
Train Epoch: 0 [560/1000 (56%)]	Loss: 0.038593
Train Epoch: 0 [640/1000 (64%)]	Loss: 0.032884
Train Epoch: 0 [720/1000 (72%)]	Loss: 0.026221
Train Epoch: 0 [800/1000 (80%)]	Loss: 0.026353
Train Epoch: 0 [880/1000 (88%)]	Loss: 0.031551
Train Epoch: 0 [960/1000 (96%)]	Loss: 0.043021
====> Epoch: 0 Average loss: 0.0509
Started training epoch no. {} 2
Train Epoch: 1 [0/1000 (0%)]	Loss: 0.027728
Train Epoch: 1 [80/1000 (8%)]	Loss: 0.032831
Train Epoch: 1 [160/1000 (16%)]	Loss: 0.019367
Train Epoch: 1 [240/1000 (24%)]	Loss: 0.021445
Train Epoch: 1 [320/1000 (32%)]	Loss: 0.025428
Train Epoch: 1 [400/1000 (40%)]	Loss: 0.019004
Train Epoch: 1 [4

In [23]:
len(bottle_neck)

1000

# Test

In [25]:
test_set = ToyDS(df[df.PI_IDS.isin(selection)])
test_loader=DataLoader(dataset= test_set, batch_size = batch_size, shuffle = True, num_workers = num_workers)

MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Female
MESH NAME NOT FOUND: Male
MESH NAME NOT FOUND: Male
MESH NAME 

In [32]:
bottle_neck=[]
for batch_idx, (data, _) in enumerate(test_loader):
            data = data.to(device, dtype=torch.float32)
            optimizer.zero_grad()
            recon_batch, mu, logvar, z= model(data)
            bottle_neck.extend(z.detach().numpy())

In [33]:
bottle_neck

[array([-0.5125515 , -0.05382395,  0.10804544,  1.1739533 ,  0.6497887 ,
        -0.2347229 , -0.58228445,  0.29939458], dtype=float32),
 array([-1.200824  , -0.5851066 , -0.25364336,  1.5224606 ,  0.06729561,
         1.28699   ,  2.2103202 ,  0.07672429], dtype=float32),
 array([ 0.93272686,  0.83364004, -1.0041033 ,  1.7578037 , -0.38320884,
         0.17935592, -0.11928867,  0.5797328 ], dtype=float32),
 array([ 0.01214548,  0.2743053 , -0.3918953 ,  0.48804986, -0.48327136,
         0.82464474, -0.3055856 , -1.1534885 ], dtype=float32),
 array([-0.79784185, -1.2829901 , -0.04676429,  0.39982447, -2.392242  ,
         0.60709727, -1.1558356 ,  0.41436872], dtype=float32),
 array([-1.3568927 ,  0.8612896 , -0.59241295,  0.53968483, -1.8831385 ,
         1.0379488 , -1.488781  ,  0.186856  ], dtype=float32),
 array([-0.9103072 ,  3.0682812 , -0.1073675 , -0.6314237 , -0.40263504,
         0.20722799,  0.8580421 , -0.0412149 ], dtype=float32),
 array([-1.093188  ,  0.9909553 , -1.6752

In [34]:
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import time


In [35]:
ds=np.array(bottle_neck)
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(ds)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

[t-SNE] Computing 71 nearest neighbors...
[t-SNE] Indexed 72 samples in 0.000s...
[t-SNE] Computed neighbors for 72 samples in 0.010s...
[t-SNE] Computed conditional probabilities for sample 72 / 72
[t-SNE] Mean sigma: 2.162510
[t-SNE] KL divergence after 250 iterations with early exaggeration: 49.802261
[t-SNE] KL divergence after 300 iterations: 1.037349
t-SNE done! Time elapsed: 0.16399407386779785 seconds


In [36]:
df_subset=pd.DataFrame()

In [41]:
df_subset['color']="black"


In [53]:
df_subset['PI']=PI_lst
df_subset.PI.unique()

array([8352708, 6913641])

In [140]:
df_subset

,tsne-2d-one,tsne-2d-two,author,color
0,15.203683,-4.184505,black,black
1,22.978985,2.953709,black,black
2,-13.019471,12.144258,black,black
3,-22.391275,14.445200,black,black
4,-7.135839,16.022161,black,black
...,...,...,...,...
67,-6.369195,-31.188566,black,black
68,17.569487,-0.229640,black,black
69,-1.176405,-1.331829,black,black
70,-4.017713,-3.593258,black,black


In [33]:
for epoch in range(epochs):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(64, 20).to(device)
            sample = model.decode(sample).cpu()

NameError: name 'train' is not defined